In [1]:
from pyspark import RDD, SparkConf, SparkContext
import os
import numpy as np
import math

from tree import Tree, TreeNode
import threading

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import time

memory = '10g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
os.environ["PYTHONHASHSEED"]=str(232)

In [2]:
NUM_WORKER = 8

def scanDB(path, seperation):
    db = []
    f = open(path, 'r')
    for line in f:
        if line:
            temp_list = line.rstrip().split(seperation)
            temp_list = [int(i) for i in temp_list]
            temp_list.sort()
            temp_list = [str(i) for i in temp_list]
            db.append(temp_list)
    f.close()
    return db

In [3]:
def runFreno(transactions, minsup):
    # for each worker: input (transactions line, minsup) and return minsup list
    
    tree = Tree(minsup)
    for trx in transactions:
        tree.insert(tree._root,trx)
    return tree.__repr__()
    
    

In [8]:
def distFreno(inFile, min_sup, sc, k):

    # Phase 1: generate Frequent items; produce vertical dataset

    # tranData is ans RDD, consider using zipwithUniqueId
    # TODO: add support for diffsets
    
    transDataRaw = scanDB(inFile, " ")
    numTrans = len(transDataRaw)
    #print(transDataRaw[:5])
    
    minsup = min_sup * numTrans
    #print("minsup", minsup)
    
    out_rdd = []
    for trx in transDataRaw:
        out_rdd.extend([trx[i:] for i in range(len(trx))])
    #print(out_rdd[:5])
    
    transDataFile = sc.parallelize(out_rdd)
    #print(transDataFile.count())
    
    transData = transDataFile.map(lambda v: (v[0], v))
    #print(transData.keys().take(5))
    transData = transData.map(lambda v: v[1])
    
    transData = transData.groupBy(lambda v: int(v[0])%k).map(lambda v : (v[0], list(v[1]))).collect()#.sortByKey()
    print(transData[0][1][:5])
    
    
    #print("transaction data num of keys:", transData.count())

    #transDataList = transData.collect()
    #print(transDataList[0])
    
    # use the configuration as the number of partitions
    print("number of partitions used: {}".format(sc.defaultParallelism))
    # itemTidsParts = itemTids.repartition(sc.defaultParallelism).glom()
    # print(itemTidsParts.take(5))

    #phase 3: EClaT from k-itemsets
    #print(transData.lookup(0).count())
    #freqItemsList = freqItemTidsList.collect()
    #freqAtoms = freqItemTidsList.keys().map(lambda k: [k]).collect()
    # print("frequent items", freqAtoms)
    #freqRange = sc.parallelize(range(0, len(freqItemsList) - 1))
    freqRange = sc.parallelize(range(0, k-1))
    freqItemsListToRun = freqRange.map(\
        lambda v: transData[v])

    #print('freqItemsListToRun', freqItemsListToRun.take(1)[0])
    
    #res = freqItemsListToRun.flatMap(lambda t: runEclat([[t[0][0]], t[0][1]], t[1], 2)).collect()
    res = freqItemsListToRun.map(lambda t: runFreno(t[1],minsup)).collect()
    return res

In [10]:
testFiles = ["retail"]
support = [0.4]

conf = SparkConf().setAppName("")
sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)
schema = StructType([
    StructField("algorithm", StringType(), False),
    StructField("datasets", StringType(), False),
    StructField("support", FloatType(), False)
])
for i in range(1):
    schema.add("test{}".format(i+1), FloatType(), True)
#experiments = []

for f in testFiles:
    for s in support:
        res = distFreno("./databases/{}.txt".format(f), s, sc, 8)
        print(res)
sc.stop()

[['8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30'], ['16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30'], ['24', '25', '26', '27', '28', '29', '30'], ['32', '33'], ['40', '41', '42', '43', '44', '45', '46', '47']]
number of partitions used: 8
["['40']", "['49']", '[]', '[]', '[]', '[]', '[]']
